In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install mysql-connector-python openpyxl

In [4]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install openpyxl xlrd pandas

In [20]:
import pandas as pd
import mysql.connector
import os
import glob

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='APSSH@8081#Root',
    database='ie_data_india'
)
cursor = conn.cursor()

folder_path = 'D:\\The World of Programming\\Github\\Data-Analysis-Import-Export-Of-India'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

def is_html_file(file_path, num_bytes=100):
    with open(file_path, 'rb') as file:
        header = file.read(num_bytes)
    return b'<html' in header or b'<!DOCTYPE html' in header

def read_file(file_path):
    try:
        if file_path.endswith('.xlsx'):
            df = pd.read_excel(file_path, engine='openpyxl')
        elif file_path.endswith('.xls'):
            df = pd.read_excel(file_path, engine='xlrd')
        elif file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        else:
            print(f"Unsupported file type: {file_path}")
            return None
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None
    return df

def create_table(df, table_name):
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

def insert_data(df, table_name):
    for _, row in df.iterrows():
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)
    conn.commit()

def sanitize_table_name(file_name):
    table_name = os.path.basename(file_name).replace(' ', '_').replace('-', '_').replace('.', '_').split('.')[0]
    return table_name[:64]  # Truncate to a maximum of 64 characters

for file_path in glob.glob(os.path.join(folder_path, '*')):
    if is_html_file(file_path):
        print(f"Skipping HTML file: {file_path}")
        continue
    
    # Skip unsupported file types
    if not file_path.endswith(('.xls', '.xlsx', '.csv')):
        print(f"Skipping unsupported file type: {file_path}")
        continue
    
    table_name = sanitize_table_name(file_path)
    df = read_file(file_path)

    if df is None or df.empty:
        print(f"Skipped file due to read error or empty file: {file_path}")
        continue

    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]
    df = df.where(pd.notnull(df), None)

    create_table(df, table_name)
    insert_data(df, table_name)

cursor.close()
conn.close()


Skipping unsupported file type: D:\The World of Programming\Github\Data-Analysis-Import-Export-Of-India\import_export_data_of_india.ipynb
